# Code embedding evaluation (315 repositories / train set)

## 1. Loading repository information

In [1]:
!pip install gdown
!gdown 1sJDBLvVRvl4Sx0ICGuoM27pREIg0M756


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.10 -m pip install --upgrade pip
Downloading...
From (uriginal): https://drive.google.com/uc?id=1sJDBLvVRvl4Sx0ICGuoM27pREIg0M756
From (redirected): https://drive.google.com/uc?id=1sJDBLvVRvl4Sx0ICGuoM27pREIg0M756&confirm=t&uuid=868e1d29-4766-48cf-af1e-0f30636bc12f
To: /Users/Henry/Documents/PyCharmProjects/RepoSim4Py/Embedding/Embedding_model_train_315/Code_embedding_evaluation_train_315/repo_info_train.pkl
100%|████████████████████████████████████████| 201M/201M [00:14<00:00, 13.8MB/s]


In [2]:
import pickle

with open("repo_info_train.pkl", "rb") as f:
    repo_info = pickle.load(f)
    f.close()

## 2. Download pre-trained model

In [4]:
!pip3 install torch
!pip3 install transformers
# !wget https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py
!curl -O https://raw.githubusercontent.com/microsoft/CodeBERT/master/UniXcoder/unixcoder.py


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10352  100 10352    0     0  20703      0 --:--:-- --:--:-- --:--:-- 20870


In [5]:
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(device)

mps


In [6]:
unixcoder_model_names = [
    "microsoft/unixcoder-base-nine",
    "microsoft/unixcoder-base",
    "microsoft/unixcoder-base-unimodal",
    "Lazyhope/unixcoder-nine-advtest",
    "Lazyhope/unixcoder-clone-detection",
    "Enoch/Unixcoder-Tuned-Code-Search-Py"
]

bert_model_names = [
    "Enoch/cocosoda-graphcodebert",
    "Enoch/graphcodebert-py"
]

In [7]:
from unixcoder import UniXcoder
from transformers import AutoTokenizer, AutoModel

unixcoder_models = {}
# Initialise unixcoder model
for u_name in unixcoder_model_names:
    unixcoder_models[u_name] = UniXcoder(u_name)
    unixcoder_models[u_name].to(device)

bert_models = {}
# Initialise GraphCodeBERT model
for b_name in bert_model_names:
    bert_models[b_name] = {}
    bert_models[b_name]["tokenizer"] = AutoTokenizer.from_pretrained(b_name, device=device)
    bert_models[b_name]["model"] = AutoModel.from_pretrained(b_name)

## 3. Generating code embeddings

In [10]:
# Getting code embeddings by UniXcoder
def get_unixcoder_embeddings(code, model):
    tokens_ids = model.tokenize([code], max_length=512, mode="<encoder-only>")
    source_ids = torch.tensor(tokens_ids).to(device)
    _, embeddings = model(source_ids)
    return embeddings

# Getting code embeddings by GraphCodeBERT
def get_bert_embeddings(code, bert_dict):
    tokenizer = bert_dict["tokenizer"]
    model = bert_dict["model"]
    inputs = tokenizer(code, padding=True, truncation=True, max_length=512, return_tensors="pt",)
    outputs = model(**inputs)
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    return embeddings

# Getting code list embeddings
def get_code_list_embeddings(data_list):
    codes_embeddings = {}
    # Using UniXcoder model
    for u_name, u_model in tqdm(unixcoder_models.items()):
        print(f" - Using {u_name} model - ")
        code_embeddings_list = torch.concat([get_unixcoder_embeddings(code, u_model) for code in data_list])
        codes_embeddings[u_name] = code_embeddings_list
    
    # Using GraphCodeBERT model
    for b_name, b_dict in tqdm(bert_models.items()):
        print(f" - Using {b_name} model - ")
        code_embeddings_list = torch.concat([get_bert_embeddings(code, b_dict) for code in data_list])
        codes_embeddings[b_name] = code_embeddings_list
    
    return codes_embeddings


# Getting embeddings
def get_embeddings(input_data=None, input_mode=None):
    if input_data is None or not input_data:
        return torch.zeros((768,), device=device)

    with torch.no_grad():
        if input_mode == "codes":
            return get_code_list_embeddings(input_data)


In [ ]:
from tqdm import tqdm

for repo_name, repo_dict in tqdm(repo_info.items()):
    print(f" - Generating code embeddings for {repo_name} - ")
    repo_dict["codes_embeddings"] = get_embeddings(input_data=repo_dict["codes"], input_mode="codes")

  0%|                                                   | 0/315 [00:00<?, ?it/s]

 - Generating code embeddings for jet-admin/jet-bridge - 



  0%|                                                     | 0/6 [00:00<?, ?it/s]

 - Using microsoft/unixcoder-base-nine model - 



 17%|███████▌                                     | 1/6 [00:53<04:27, 53.59s/it]

...

 - Generating code embeddings for keras-team/keras - 



  0%|                                                     | 0/6 [00:00<?, ?it/s]

 - Using microsoft/unixcoder-base-nine model - 



 17%|███████                                   | 1/6 [13:49<1:09:09, 829.91s/it]

 - Using microsoft/unixcoder-base model - 


## 4. Saving code embeddings

In [ ]:
save_device = "cpu"
# Change the tensor format to CPU
for repo_name, repo_dict in repo_info.items():
    if type(repo_dict["codes_embeddings"]) is torch.Tensor:
        repo_info[repo_name]["codes_embeddings"] = repo_info[repo_name]["codes_embeddings"].to(save_device)
    else:
        for model_name, model_tensor in repo_dict["codes_embeddings"].items():
            repo_info[repo_name]["codes_embeddings"][model_name] = model_tensor.to(save_device)

In [ ]:
with open("repo_info_train_code_embeddings.pkl", "wb") as f:
    pickle.dump(repo_info, f)
    f.close()

## 5. Calculating similarity

In [ ]:
from torch.nn import CosineSimilarity
from itertools import combinations
import pandas as pd

cossim = CosineSimilarity(dim=0, eps=1e-8)
res = []
num_of_repos = len(repo_info)
num_of_rows = num_of_repos * (num_of_repos - 1) // 2

model_names = unixcoder_model_names + bert_model_names

for repo1, repo2 in tqdm(combinations(repo_info.keys(), 2), total=num_of_rows):
    row = [repo1, repo2, repo_info[repo1]["topic"], repo_info[repo2]["topic"]]
    for model_name in model_names:

        if type(repo_info[repo1]["codes_embeddings"]) is torch.Tensor:
            embedding1 = repo_info[repo1]["codes_embeddings"]
        else:
            embedding1 = torch.mean(repo_info[repo1]["codes_embeddings"][model_name], dim=0)

        if type(repo_info[repo2]["codes_embeddings"]) is torch.Tensor:
            embedding2 = repo_info[repo2]["codes_embeddings"]
        else:
            embedding2 = torch.mean(repo_info[repo2]["codes_embeddings"][model_name], dim=0)

        similarity = cossim(embedding1, embedding2).cpu().detach().numpy().item()
        row.append(similarity)

    res.append(row)

df = pd.DataFrame(res, columns=["repo1", "repo2", "topic1", "topic2"] + model_names)
df

In [ ]:
df.dropna(inplace=True, subset=model_names)
df.reset_index(drop=True, inplace=True)
df

In [ ]:
df.to_csv("code_embedding_evaluation_train_315.csv")

## 6. Evaluating different models

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

model_auc = {}
y_true = df['topic1'] == df['topic2']
fig = plt.figure(figsize=(12, 10))


def roc(model_name):
    y_score = df[model_name]
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    roc_auc = auc(fpr, tpr)
    model_auc[model_name] = roc_auc
    plt.plot(fpr, tpr, label=model_name)


for model in model_names:
    roc(model)

# Order labels by each model's auc score
handles, labels = plt.gca().get_legend_handles_labels()
order = sorted(range(len(labels)), key=lambda k: -model_auc[labels[k]])
ordered_handles = [handles[idx] for idx in order]
ordered_labels = [f'{labels[idx]} (AUC = {model_auc[labels[idx]]:.3f})' for idx in order]
plt.legend(ordered_handles, ordered_labels, loc="lower right")

plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'Receiver Operating Characteristic of different models')
plt.savefig('roc_code_evaluation_train_315.eps')
plt.show()

In [ ]:
model_auc